In [6]:
# Install newer version of Pytorch
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

# Cloning ONLY colab-integration branch
from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Githyb password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone -b colab-integration --single-branch "https://$GITHUB_AUTH@github.com/baltekgajda/LegoSortingRecognition.git"

# Navigate to project folder
os.chdir("./LegoSortingRecognition")

# Create dir for models
!mkdir models

Github user··········
Githyb password··········
Cloning into 'LegoSortingRecognition'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 78 (delta 34), reused 56 (delta 18), pack-reused 0
Unpacking objects: 100% (78/78), done.


In [0]:
# Import necessary libraries
from __future__ import print_function
from __future__ import division
from google.colab import drive
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import utils

from data_loader import load_data
from feature_extraction import initialize_model, train_model
import VGGFactory

In [8]:
# Mount Google Drive to access data
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def get_params_to_update(model):
  params_to_update = model.parameters()
  params_to_update = []
  for name, param in model.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        
  return params_to_update

In [0]:
def train_and_evaluate_classifier_only(dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  model = VGGFactory.create_model(1, num_of_classes)
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-classifier-only")
  return (model_ft, hist)

In [0]:
def train_and_evaluate_last_conv(dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  model = VGGFactory.create_model(2, num_of_classes)
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-last_conv")
  return (model_ft, hist)

In [0]:
def train_and_evaluate_full_net(dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  model = VGGFactory.create_model(3, num_of_classes)
  
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-full-net")
  return (model_ft, hist)

In [0]:
def train_and_evaluate_simplified_net(model, dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  simpler_model = VGGFactory.simplify_model(model)
  
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-simplified")
  return (model_ft, hist)

In [16]:
NUM_OF_CLASSES = 20
NUM_OF_EPOCHS = 1
INPUT_SIZE = 224
DATA_DIR = '/content/drive/My Drive/Studia/SNR/lego/'

print("Initializing Datasets and Dataloaders...")
dataloaders_dict = load_data(DATA_DIR, INPUT_SIZE, batch_size=32, num_workers=4)


if (torch.cuda.is_available() == False):
  raise RuntimeError("GPU is not available!")   
device = torch.device("cuda:0")

torch.cuda.current_device()


m_classifier, hist_classifier = train_and_evaluate_classifier_only(dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
m_last_conv, hist_last_conv = train_and_evaluate_last_conv(dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
m_on_full_net, hist_on_full_net = train_and_evaluate_full_net(dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
m_simplified, hist_simplified = m_train_and_evaluate_simplified_net(model_on_full_net, dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)

Initializing Datasets and Dataloaders...
Epoch 0/0
----------
train Loss: 2.1584 Acc: 0.3462
Param:  weight , values:  Parameter containing:
tensor([[-1.9490e-03, -1.1519e-02,  7.8316e-03,  ...,  1.5645e-02,
          9.4340e-03, -8.7332e-03],
        [-1.3435e-02,  5.8485e-04,  1.5262e-02,  ..., -2.1201e-03,
          6.3542e-03,  1.0188e-02],
        [ 2.0899e-03, -1.0419e-02,  4.9655e-03,  ...,  6.0938e-03,
          7.7213e-03, -4.6346e-03],
        ...,
        [-9.0351e-03,  1.2296e-02, -1.3158e-02,  ..., -1.1404e-02,
         -9.6645e-03, -9.9509e-03],
        [-4.5578e-03, -1.1931e-02,  4.9020e-03,  ..., -5.3374e-03,
         -1.5376e-02,  1.2703e-02],
        [-6.0301e-03, -5.0445e-04, -3.9816e-03,  ..., -7.1132e-05,
          6.7944e-05, -6.4846e-03]], device='cuda:0', requires_grad=True)
Param:  bias , values:  Parameter containing:
tensor([ 0.0136, -0.0031, -0.0004,  0.0043,  0.0092, -0.0140, -0.0158,  0.0061,
        -0.0091, -0.0078,  0.0017,  0.0124, -0.0138, -0.0100,  0

(VGG(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): ReLU(inplace)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (4): ReLU(inplace)
     (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (7): ReLU(inplace)
     (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (9): ReLU(inplace)
     (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (12): ReLU(inplace)
     (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (14): ReLU(inplace)
     (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (16): Co

In [0]:
import os
import shutil

os.chdir("content")
shutil.rmtree('./LegoSortingRecognition')

In [0]:
# num_classes = 20
# num_of_epochs = 50
# input_size = 224

# model = VGGFactory.create_model(1, num_classes)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)

# print()

# simple_model = VGGFactory.simplify_model(model)
# for name, param in simple_model.named_parameters():
#     if param.requires_grad:
#         print(name)

# print("Initializing Datasets and Dataloaders...")
# dataloaders_dict = load_data(data_dir, input_size, 0.3, 0.1, 0.1)

# params_to_update = model.parameters()
# print("Params to learn:")
# params_to_update = []
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         params_to_update.append(param)
#         print("\t", name)

# optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# # Setup the loss fxn
# criterion = nn.CrossEntropyLoss()

# # Detect if we have a GPU available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# # Necessary to run script on Windows
# torch.cuda.current_device()

# # Train and evaluate
# model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_of_epochs)

# # Save trained model
# utils.save_model(model_ft, "./models")
